# EDA for Data Incubator project proposal, fall 2020

## How does performance relate to compensation in publicly funded universities?

### -1- Get data

#### Start with the raw csv file downloaded from website

Link to [Urban Institute's data explorer](https://educationdata.urban.org/data-explorer/colleges/) (super cool). 

Link to [my document](https://github.com/dagny099/does_good_payoff/blob/master/docs/getting-started.rst) showing selection criteria and variables.

*Rounds of downloads* <br>
Rd_1: TX, FL         ... Approximate results: 188k records from 838 institutions <br>
Rd_2: OR, AZ, MI, OH ... Approximate results: 169k records from 752 institutions <br>
Rd_3: NY, GA, MN     ... Approximate results: 170k records from 755 institutions <br>
Rd_4: CA             ... Approximate results: 164k records from 729 institutions <br>
Rd_5: VA, PA, IN, WI ... Approximate results: 178k records from 791 institutions <br>

Note: There appears to be a query limit of around 200k records, hence multiple rounds of downloads. 

Rationale behind states chosen:
- Question of interest is geared towards public funding and higher education outcomes
- Wikipedia lists the Top 10 university campuses by enrollment, by year
- Rd 1. I chose to include the states where those campuses are located (based on [2018–19 academic year]{https://en.wikipedia.org/wiki/List_of_United_States_public_university_campuses_by_enrollment#2018%E2%80%9319_enrollment})
- Rd 2-4. Include more geographic diversity by adding CA, NY, OR, MI
- Rd 5. Include any state with a university in the top-10 since 2009, added IN, PA

Manually summed size of aggregated csv files: **70MB**

In [ ]:
# Use bash to combine all files with same columns from multiple downloads:
 
!(head -1 ../../EducationDataPortal_TX_FL_years_after_entry.csv && tail -n +2 -q ../../EducationDataPortal*_years_after_entry.csv ) > ../../EducationDataPortal_years_after_entry_ALL.csv
!(head -1 ../../EducationDataPortal_TX_FL_level_of_study.csv && tail -n +2 -q ../../EducationDataPortal*_level_of_study.csv ) > ../../EducationDataPortal_level_of_study_ALL.csv
!(head -1 ../../EducationDataPortal_TX_FL_institutions.csv && tail -n +2 -q ../../EducationDataPortal*_institutions.csv ) > ../../EducationDataPortal_institutions_ALL.csv

!mv ../../EducationDataPortal_years_after_entry_ALL.csv ../data/raw/higherEd/usa/
!mv ../../EducationDataPortal_level_of_study_ALL.csv ../data/raw/higherEd/usa/
!mv ../../EducationDataPortal_institutions_ALL.csv ../data/raw/higherEd/usa/



In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline

pathDir = '../data/raw/higherEd/usa/'
filename_inst = 'EducationDataPortal_institutions_ALL.csv'
filename_los = 'EducationDataPortal_level_of_study_ALL.csv'
filename_yae = 'EducationDataPortal_years_after_entry_ALL.csv'

# Read downloaded csv files
tmp = pd.read_csv(pathDir+filename_inst)
tmp_los = pd.read_csv(pathDir+filename_los)
tmp_yae = pd.read_csv(pathDir+filename_yae)

### -2- Reduce data, Clean data

In [ ]:
# I know I want to look at Degree-granting institutions but let's see what other inst_category values there are:
want='Degree-granting, primarily baccalaureate or above'
print(f"There are {tmp[tmp.inst_category==want].shape[0]} records from {want} institutions")
print(f"# which is {round(tmp[tmp.inst_category==want].shape[0]/tmp.shape[0],2)} of all {tmp.shape[0]} records\n")

print(tmp.inst_category.value_counts())


In [ ]:
# Let's look at the size of those institutions:
tmp[tmp.inst_category==want]['inst_size'].value_counts()

In [ ]:
# How many are institutions with 5K or above (that's 3 categories):
sizeFilt=tmp.apply(lambda row: row['inst_size'] in ['5,000-9,999','10,000-19,999','20,000 and above'], axis=1)

In [ ]:
tmp[sizeFilt].groupby(['state_name','year'])['number_enrolled_total'].count()

In [ ]:
tmp[sizeFilt].pivot(index='unitid',columns='year',values='number_enrolled_total')

In [ ]:
# ------ REDUCE ROWS TO SUIT QUESTIONS OF INTEREST ------
criteria = dict({'inst_category':'Degree-granting, primarily baccalaureate or above',
                 'inst_size':['20,000 and above','10,000-19,999']})

univData = tmp[(tmp['inst_category']==criteria['inst_category'])] # & (tmp['inst_size']==criteria['inst_size'])]
univData = univData[(univData.inst_size=='20,000 and above')|(univData.inst_size=='10,000-19,999')]
univData['inst_category'].value_counts()
print(f"Reduced data to suit questions-of-interest:")
print(f"ORIG: {tmp.shape[0]} records from {tmp.unitid.nunique()} institutions\nto")
print(f"FILTERED: {univData.shape[0]} records from {univData.unitid.nunique()} institutions")
print(f"Data from {univData.state_name.nunique()} states")
print(f"\nThese were the criteria applied:")
print(criteria)

In [ ]:
# Which institutions have at least 8 years of data? => Generate a list of schools (unitid)
# Enrollment data, e.g. "number_enrolled_total"
nYrs_ts = 8

# using pivot -- I don't think I did this correctly ...

# df_AvgEnrollment = pd.pivot_table(univData,index=['state_name','unitid'],values='number_enrolled_total',\
#                aggfunc=({np.mean,'count'})).sort_values(['state_name','mean'],ascending=False)
# df_AvgEnrollment = df_AvgEnrollment[df_AvgEnrollment['count']>=nYrs_ts]

Use_These_Univ = univData.reset_index().unitid.unique()

In [ ]:
# Keep columns with at least 80% non-null values
keepThresh = .80
keepCols = [c for c in dataBigUniv.columns if dataBigUniv[c].isnull().sum() < (dataBigUniv.shape[0]*keepThresh)]

print(f'Dropped these columns for missing more than {100*keepThresh}% values:')
print([col for col in dataBigUniv.columns if col not in keepCols])

In [ ]:
# Filter all data files to only include institutions that meet criteria
tmp['keepRow']=tmp['unitid'].apply(lambda x: True if x in Use_These_Univ else False)
dataInst = tmp[tmp['keepRow']==True]
dataInst.drop('keepRow',axis=1,inplace=True)

tmp_los['keepRow']=tmp_los['unitid'].apply(lambda x: True if x in Use_These_Univ else False)
dataLOS = tmp_los[tmp_los['keepRow']==True]
dataLOS.drop('keepRow',axis=1,inplace=True)

tmp_yae['keepRow']=tmp_yae['unitid'].apply(lambda x: True if x in Use_These_Univ else False)
dataYAE = tmp_yae[tmp_yae['keepRow']==True]
dataYAE.drop('keepRow',axis=1,inplace=True)


In [ ]:
# Merge data sets  (would definitely create SQL tables for this)
data = pd.merge(dataInst, dataYAE, how ='left', on =['unitid','year','inst_name','state_name'])
data = pd.merge(data, dataLOS, how ='left', on =['unitid','year','inst_name','state_name'])
data.describe(include='all')


In [ ]:
# KEEP AN EXCEL FILE WITH ALL DATA
# why? I need to double check that what I'm doing in Python is correct

data.to_csv('../data/processed/'+'Merged_Univ_DataMore3.csv')


In [ ]:
# CHANGE SOME DATATYPES:
# Change dtype 'year' to DATETIME64
# Change dtype 'unitid', 'inst_name', 'state_name' as CATEGORY

dataInst['year'] = dataInst['year'].apply(pd.to_datetime, format='%Y')



In [ ]:
# Add columns for AdmissionRate and EnrollmentRate
univData['admission_rate']=univData['number_admitted']/univData['number_applied']
univData['enrollment_rate']=univData['number_enrolled_total']/univData['number_admitted']


In [ ]:
pathDir = '../data/processed/'
filename = 'EducationDataPortal_HigherEdMore.xlsx' #This started as an empty excel file with data dictionary


# Read excel sheet
whichSheet = 'Data_by_Institution'
df = pd.read_excel(pathDir+filename, sheet_name=whichSheet, 
                   usecols=keepCols, na_values=np.nan, verbose=True,
                   dtype={'unitid':'category', 'inst_name': 'category', 'state_name': 'category'},
                   parse_dates=['year'])

whichSheet = 'Breakdown_years_after_entry'
df_yae = pd.read_excel(pathDir+filename, sheet_name=whichSheet, 
                   na_values=np.nan, verbose=True,
                   dtype={'unitid':'category', 'inst_name': 'category', 'state_name': 'category'},
                   parse_dates=['year'])

whichSheet = 'Breakdown_level_of_study'
df_los = pd.read_excel(pathDir+filename, sheet_name=whichSheet, 
                   na_values=np.nan, verbose=True,
                   dtype={'unitid':'category', 'inst_name': 'category', 'state_name': 'category'},
                   parse_dates=['year'])


In [ ]:
# Merge data sets  (would definitely create SQL tables for this)
data = pd.merge(df, df_yae, how ='inner', on =['unitid','year','inst_name','state_name'])
data = pd.merge(data, df_los, how ='inner', on =['unitid','year','inst_name','state_name'])
data.to_csv('../data/processed/'+'Merged_Univ_Data.csv')
data.describe(include='all')


### -3- Compute some stats, Group data

In [ ]:
# Add column for admission_rate
data['admission_rate'] = data['number_admitted'] /  data['number_applied']


In [ ]:
# Enrollment & Admissions Trends
cols = ['inst_name','state_name','number_applied','number_admitted','number_enrolled_total','admission_rate']
df_Enrollment = pd.DataFrame(data.groupby(['state_name','inst_name','year'])[cols].mean().to_records()) 


In [ ]:
# Graduation Trends
cols = ['inst_name','state_name','completers_150pct','completion_rate_150pct']
df_Grad = pd.DataFrame(data.groupby(['state_name','inst_name','year'])[cols].mean().to_records()) 


In [ ]:
# Funding Trends, Revenue
cols = ['inst_name','state_name','rev_tuition_fees_net','rev_tuition_fees_gross']
df_Rev = pd.DataFrame(data.groupby(['state_name','inst_name','year'])[cols].mean().to_records()) 

In [ ]:
# Funding Trends, Expenditures
cols = ['inst_name','state_name','exp_total_current','exp_total_salaries','exp_total_benefits']
df_Exp = pd.DataFrame(data.groupby(['state_name','inst_name','year'])[cols].mean().to_records()) 


In [ ]:
# Earnings Trends
# cols = ['inst_name','state_name','earnings_mean','count_working','count_not_working']
# df_Earnings = pd.DataFrame(data.groupby(['state_name','inst_name','year'])[cols].mean().to_records()) 

### -5- Have a look at the data, visually and export table

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

figpathDir = '../src/visualization/'

In [ ]:
# PAIR-PLOT for ENROLLMENT TRENDS: 
DF = df_Enrollment
sns.set_style('white')

pair_plot = sns.pairplot(DF, hue='year');
pair_plot.savefig(figpathDir+'Enrollment_pair_plot_by_YEAR.png')  

pair_plot = sns.pairplot(DF, hue='state_name');
pair_plot.savefig(figpathDir+'Enrollment_pair_plot_by_STATE.png')  


In [ ]:
# PAIR-PLOT for GRADUATION TRENDS: 
DF = df_Grad
sns.set_style('white')

pair_plot = sns.pairplot(DF, hue='year');
pair_plot.savefig(figpathDir+'Grad_pair_plot_by_YEAR.png')  

pair_plot = sns.pairplot(DF, hue='state_name');
pair_plot.savefig(figpathDir+'Grad_pair_plot_by_STATE.png')  


In [ ]:
# PAIR-PLOT for Funding-Revenue TRENDS: 
DF = df_Rev
sns.set_style('white')

pair_plot = sns.pairplot(DF, hue='year');
pair_plot.savefig(figpathDir+'Fund_Rev_pair_plot_by_YEAR.png')  

pair_plot = sns.pairplot(DF, hue='state_name');
pair_plot.savefig(figpathDir+'Fund_Rev_pair_plot_by_STATE.png')  


In [ ]:
# PAIR-PLOT for Funding-Expenditures TRENDS: 
DF = df_Exp
sns.set_style('white')

pair_plot = sns.pairplot(DF, hue='year');
pair_plot.savefig(figpathDir+'Fund_Exp_pair_plot_by_YEAR.png')  

pair_plot = sns.pairplot(DF, hue='state_name');
pair_plot.savefig(figpathDir+'Fund_Exp_pair_plot_by_STATE.png')  


#### Time series of admission_rate

In [ ]:
sns.set()
pvDf_admission_rate = df_Enrollment['']
ad_ts = pvDf_admission_rate.plot(figsize=(12,8),lw=2,title='Admission Rate at Public Universities over Time')
ad_ts.legend(loc='center left', bbox_to_anchor=(1.25, 0.5), ncol=1)
plt.ylabel('number admissions / number applicants');

In [ ]:
pd.pivot(df_Enrollment

In [ ]:
#### Boxplots
sns.boxplot()

In [2]:
# --------------------------------
# IMPORT MODULES
# --------------------------------
import pandas as pd
import numpy as np
from datetime import datetime as dt

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

# Prune as necessary:
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
import chart_studio.plotly as py
import plotly
import cufflinks as cf

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# Create a variable, options, with a list of dicts w/ keys {'label': , 'value'}
def make_options(df):
    n=df.unitid.nunique()
    state_options=[{'label': 'All '+str(n)+" schools", 'value':''}]
    for state in df.state_name.unique():
        n=df.groupby(['state_name'])['number_applied'].count().loc[state]
        state_options.append({'label': state+" ("+str(n)+" schools)", 'value': state })
    return state_options


In [ ]:
# Return 
def sem_btwn(x):
    return round(np.std(x)/np.sqrt(x.count()),3)


In [4]:
# --------------------------------
# Acquire the Data:
# --------------------------------
path2file="/Users/bhs/PYTHON-STUFF/DataIncubator/does_good_payoff/data/interim/analyzeMe_n175.csv"
df = pd.read_csv(path2file, na_values=np.nan,#parse_dates=['year'], 
                   dtype={'unitid':'category', 'inst_name': 'category', 'state_name': 'category',
                         'enrollement_rate': 'float64', 'female_pct': 'float64', 'married_pct': 'float64'})

# --------------------------------
# Data Prune:
# --------------------------------
# Drop years earlier than 2001 (availability of key measures)
df = df[df.year>=2001];

# Set 'year' as datetime 
df['year'] = df['year'].apply(pd.to_datetime, format='%Y')

# Keep these columns (subset from CSV) as potential features for model:
keepcols = ['admission_rate','enrollement_rate','number_applied','number_admitted','number_enrolled_total',
        'rev_tuition_fees_gross', 'rev_tuition_fees_net','rev_total_current','rev_fed_approps_grants','rev_state_local_approps_grants','rev_other',
       'exp_total_current','exp_instruc_total','exp_acad_supp_total','exp_student_serv_total','exp_res_pub_serv_total',
        'completers_150pct','completion_rate_150pct','female_pct','married_pct',
      'year','unitid','inst_name','state_name']

dropcols = [c for c in df.columns if c not in keepcols]

df.drop(dropcols,axis=1,inplace=True)


In [5]:
# --------------------------------
# GOAL: Want a Balanced dataset w XYZ schools over 17 yrs 
# --------------------------------

# ENROLLMENT SECTION
nYrs = 17
which_columns = ['enrollement_rate']
tmpDf = df.groupby('unitid')[which_columns].count()
# Identify schools with data in all years, only include those:
unitids = tmpDf[tmpDf[which_columns[0]]==nYrs].index.to_list()
filt = df.apply(lambda row: row['unitid'] in unitids, axis=1)
# Make a DF & list of options for the dropdown menu - ENROLLMENT SECTION
df = df[filt]  
state_options_enrollment = make_options(df)

# FINANCE SECTION
#lose 9 out of the 170+ schools who didn't have revenue data
# After dropping those schools, all measures have same # records 
#   EXCEPT there are 18 records missing rev_tuition_fees_net  (go back and fill with avg)
nYrs= 17
which_columns = ['rev_total_current','exp_total_current','rev_tuition_fees_gross','rev_tuition_fees_net','exp_instruc_total']
tmpDf = df.groupby(['unitid'])[which_columns].count()
# Identify schools with data in all years, only include those:
unitids = tmpDf[tmpDf[which_columns[0]]==nYrs].index.to_list()
filt = df.apply(lambda row: row['unitid'] in unitids, axis=1)
dfFin1 = df[filt]
state_options_finance = make_options(dfFin1)

In [6]:
# DEFINE PLOTTING COLORS & LABELS FOR CONSISTENTLY GRAPHING SERIES
seriez = {'number_applied': {'color': '#F44DDB', 'label': "Number of student applications"},
            'number_admitted': {'color': '#CF1214', 'label': "Number of students admitted"},
            'number_enrolled_total': {'color': '#0E3DEC', 'label': "Number enrolled"},
            'admission_rate': {'color': '#CF1214', 'label': "Admission rate (# applications/# admissions)"},
            'enrollement_rate': {'color': '#0E3DEC', 'label': "Enrollment rate (# admissions/# enrolled"},
        }
        

In [55]:
# Background 1 Callback
active_tab='CumulativeAdmissions'
selected=''

# Set xlabel based on selection (useful for debuggins)
xlab=[opt['label'] for opt in state_options_enrollment if opt['value']==selected]
    
    # Render the tab content based on value of 'active_tab' (other input, 'selected' used to crossfilter)
if active_tab == "CumulativeAdmissions":
    which_columns = ['number_applied','number_admitted','number_enrolled_total']
    graph_title = 'Enrollment in College Fails to Keep Pace with Admissions'+": "+xlab[0]
    ylabel = 'Total Number of Students'        
    markdown_comments = """Insert comments A, B, C"""
    if len(selected)==0:
        df_fig = df.groupby(['year'])[which_columns].aggregate([('Sum','sum'), ('Nschools','count')])
    else:
        df_fig = df[df.state_name==selected].groupby(['year'])[which_columns].aggregate([('Sum','sum'), ('Nschools','count')])
    fig = go.Figure()
    for col in which_columns:
        fig.add_trace(
            go.Scatter(x=df_fig.index, y=df_fig[col]['Sum'], 
                name = seriez[col]['label'], marker_color='rgba(152, 0, 0, .8)',
                line = dict(color = seriez[col]['color']), opacity = 0.8))

elif active_tab == "RateAdmissions":
    which_columns = ['admission_rate','enrollement_rate']
    graph_title = 'Admission and Enrollment Rates over time'+": "+xlab[0]
    ylabel = 'Rate'
    markdown_comments = """Insert comments D, E"""
    if len(selected)==0:
        df_fig = df.groupby(['year'])[which_columns].aggregate([('Avg',np.mean), ('stdev',np.std), ('Nschools','count'), ('SEM', sem_btwn)])        
    else:
        df_fig = df[df.state_name==selected].groupby(['year'])[which_columns].aggregate([('Avg',np.mean), ('stdev',np.std), ('Nschools','count'), ('SEM', sem_btwn)])
    fig = go.Figure()
    for col in which_columns:
        fig.add_trace(
            go.Scatter(x=df_fig.index, y=df_fig[col]['Avg'], 
                name = seriez[col]['label'], marker_color='rgba(152, 0, 0, .8)',
                line = dict(color = seriez[col]['color']), 
                error_y = dict(type='data', array=df_fig[col]['SEM'], visible=True),
                opacity = 0.8))
# Set options common to all traces with fig.update_traces
fig.update_traces(mode='lines+markers', marker_line_width=2, marker_size=10)
fig.update_layout(title=graph_title,yaxis={'title': ylabel},
                  yaxis_zeroline=True, xaxis_zeroline=True)
fig.show()


In [60]:
for c1 in df_fig.columns[]

MultiIndex(levels=[['number_applied', 'number_admitted', 'number_enrolled_total'], ['Sum', 'Nschools']],
           codes=[[0, 0, 1, 1, 2, 2], [0, 1, 0, 1, 0, 1]])

In [80]:
df_fig.columns[0][0]

'number_applied'

In [81]:
blah = df_fig.columns
blah

MultiIndex(levels=[['number_applied', 'number_admitted', 'number_enrolled_total'], ['Sum', 'Nschools']],
           codes=[[0, 0, 1, 1, 2, 2], [0, 1, 0, 1, 0, 1]])

In [93]:
blah.levels[1].to_

Index(['Sum', 'Nschools'], dtype='object')